In [ ]:
import os
import pandas as pd
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from joblib import load

from tensorflow.keras.models import load_model

## VGG19

In [ ]:
#model_image = load_model(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_image_hybride_VGG19.h5'))
#model_lstm_1 = load_model(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_audio_hybride_VGG19.h5'))
#model_combined = load_model(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_combined_hybride_VGG19.h5'))

## INCEPTION

```
# Ce texte est au format code
```



In [ ]:
#model_image = load_model(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_image_hybride_inceptionv3.h5'))
#model_lstm_1 = load_model(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_audio_hybride_inceptionv3.h5'))
#model_combined = load_model(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_combined_hybride_inceptionv3.h5'))

## Resnet

In [ ]:
#model_image = load_model(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_image_hybride_resnet.h5'))
#model_lstm_1 = load_model(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_audio_hybride_resnet.h5'))
#model_combined = load_model(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_combined_hybride_resnet.h5'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # Monte Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importation des données

Caractéristiques des images

In [ ]:
path = '/content/drive/My Drive/Multimodale/Data_Features/'

In [ ]:
images_features = pd.read_csv(os.path.join(path, 'images_features.csv'))
#images_features = pd.read_csv(os.path.join(path, 'images_features_inception.csv'))
#images_features = pd.read_csv(os.path.join(path, 'images_features_resnet.csv'))

In [ ]:
images_features.head(5)

,SubClass,IMAGE,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,...,C_4086,C_4087,C_4088,C_4089,C_4090,C_4091,C_4092,C_4093,C_4094,C_4095
0,beach,beach0.png,0.538444,0.0,0.463963,0.028087,1.296627,0.921912,0.0,0.0,...,0.038778,0.904302,0.992052,0.869632,0.0,0.0,0.0,0.0,0.325832,0.090953
1,beach,beach1.png,0.478749,0.0,0.473210,0.054322,1.282259,0.920567,0.0,0.0,...,0.085440,0.813594,0.995918,0.787818,0.0,0.0,0.0,0.0,0.302753,0.100030
2,beach,beach10.png,0.385724,0.0,0.556242,0.000000,1.229769,0.983087,0.0,0.0,...,0.152053,0.868707,1.031767,0.771014,0.0,0.0,0.0,0.0,0.240575,0.052493
3,beach,beach100.png,0.518872,0.0,0.564591,0.000000,1.280057,0.901700,0.0,0.0,...,0.062651,0.821810,1.018209,0.728661,0.0,0.0,0.0,0.0,0.272786,0.080897
4,beach,beach101.png,0.549013,0.0,0.554230,0.000000,1.282579,0.902930,0.0,0.0,...,0.069760,0.803324,1.024356,0.739038,0.0,0.0,0.0,0.0,0.264427,0.075939


Les Mfcc

In [ ]:
audios_features = pd.read_csv(os.path.join(path, 'audios_features.csv'))

In [ ]:
audios_features.head(5)

,Label,SubClass,IMAGE,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63
0,FOREST,forest,forest0.png,0.098722,0.000000,0.0,0.266995,0.0,0.604112,0.0,...,0.083809,0.000000,0.155769,0.000000,0.392649,0.404254,0.335206,0.180659,0.000000,0.358419
1,FOREST,forest,forest1.png,0.519325,0.000000,0.0,0.356544,0.0,1.147089,0.0,...,0.000000,0.504670,0.405955,0.467922,0.344288,0.093796,0.738429,0.101215,0.000000,0.041580
2,FOREST,forest,forest2.png,0.000000,0.401638,0.0,0.640012,0.0,0.228058,0.0,...,0.103851,0.000000,0.000000,0.000000,0.959596,1.090038,0.371306,0.000000,0.000000,0.662682
3,FOREST,forest,forest3.png,0.000000,0.000000,0.0,0.688969,0.0,0.949568,0.0,...,0.400116,0.000000,0.042132,0.000000,0.216583,0.391978,0.000000,0.000000,0.000000,0.340741
4,FOREST,forest,forest4.png,0.342916,0.000000,0.0,0.096814,0.0,0.315124,0.0,...,0.000000,0.016161,0.292937,0.000000,0.066487,0.000000,0.447823,0.000000,0.203651,0.499455


### Division des données

- L'objectif est d'avoir 2/3 des données ( auditives et visuelles) pour l'entrainement des modèles, et 1/3 pour leur test.
- Après les 2/3 sera diviser sur 2, pour que la moitié être pour l'entrainement de (réseau de neurones dense sur les données auditives et un autre réseau sur les données visuelles) et l'autre moitié pour entrainer un autre modèle sur son combinaison ( combinaison des données auditives et visuelles)

1. Fusion des données

In [ ]:
images_features.sort_values(by=['SubClass', 'IMAGE'], ascending=[True, True], inplace=True)
audios_features.sort_values(by=['SubClass', 'IMAGE'], ascending=[True, True], inplace=True)

In [ ]:
images_features.head(5)

,SubClass,IMAGE,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,...,C_4086,C_4087,C_4088,C_4089,C_4090,C_4091,C_4092,C_4093,C_4094,C_4095
0,beach,beach0.png,0.538444,0.0,0.463963,0.028087,1.296627,0.921912,0.0,0.0,...,0.038778,0.904302,0.992052,0.869632,0.0,0.0,0.0,0.0,0.325832,0.090953
1,beach,beach1.png,0.478749,0.0,0.473210,0.054322,1.282259,0.920567,0.0,0.0,...,0.085440,0.813594,0.995918,0.787818,0.0,0.0,0.0,0.0,0.302753,0.100030
2,beach,beach10.png,0.385724,0.0,0.556242,0.000000,1.229769,0.983087,0.0,0.0,...,0.152053,0.868707,1.031767,0.771014,0.0,0.0,0.0,0.0,0.240575,0.052493
3,beach,beach100.png,0.518872,0.0,0.564591,0.000000,1.280057,0.901700,0.0,0.0,...,0.062651,0.821810,1.018209,0.728661,0.0,0.0,0.0,0.0,0.272786,0.080897
4,beach,beach101.png,0.549013,0.0,0.554230,0.000000,1.282579,0.902930,0.0,0.0,...,0.069760,0.803324,1.024356,0.739038,0.0,0.0,0.0,0.0,0.264427,0.075939


In [ ]:
audios_features.head(5)

,Label,SubClass,IMAGE,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63
439,BEACH,beach,beach0.png,0.473053,0.000000,0.0,0.000000,0.0,0.537345,0.0,...,0.335376,0.000000,0.000000,0.014135,0.672928,0.125935,0.210799,0.000000,0.000000,0.169178
440,BEACH,beach,beach1.png,0.201572,0.000000,0.0,0.296858,0.0,0.703890,0.0,...,0.000000,0.069877,0.120306,0.216792,0.945109,0.000000,0.828891,0.000000,0.134276,0.117310
449,BEACH,beach,beach10.png,0.451867,0.474174,0.0,0.376022,0.0,0.319420,0.0,...,0.000000,0.179926,0.000000,0.038148,0.976591,0.114330,0.885398,0.000000,0.000000,0.170671
539,BEACH,beach,beach100.png,0.321822,0.298498,0.0,0.000000,0.0,0.713756,0.0,...,0.074421,0.418189,0.533106,0.235142,0.798533,0.217088,0.600934,0.000000,0.000000,0.207440
540,BEACH,beach,beach101.png,0.497097,0.000000,0.0,0.000000,0.0,0.429101,0.0,...,0.000000,0.664844,0.000000,0.000000,0.269964,0.023534,0.596322,0.155184,0.000000,0.211958


In [ ]:
colonne = audios_features.pop('Label')
audios_features['Label'] = colonne

In [ ]:
audios_features.head(5)

,SubClass,IMAGE,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,Label
439,beach,beach0.png,0.473053,0.000000,0.0,0.000000,0.0,0.537345,0.0,0.0,...,0.000000,0.000000,0.014135,0.672928,0.125935,0.210799,0.000000,0.000000,0.169178,BEACH
440,beach,beach1.png,0.201572,0.000000,0.0,0.296858,0.0,0.703890,0.0,0.0,...,0.069877,0.120306,0.216792,0.945109,0.000000,0.828891,0.000000,0.134276,0.117310,BEACH
449,beach,beach10.png,0.451867,0.474174,0.0,0.376022,0.0,0.319420,0.0,0.0,...,0.179926,0.000000,0.038148,0.976591,0.114330,0.885398,0.000000,0.000000,0.170671,BEACH
539,beach,beach100.png,0.321822,0.298498,0.0,0.000000,0.0,0.713756,0.0,0.0,...,0.418189,0.533106,0.235142,0.798533,0.217088,0.600934,0.000000,0.000000,0.207440,BEACH
540,beach,beach101.png,0.497097,0.000000,0.0,0.000000,0.0,0.429101,0.0,0.0,...,0.664844,0.000000,0.000000,0.269964,0.023534,0.596322,0.155184,0.000000,0.211958,BEACH


In [ ]:
label_encoder = LabelEncoder()

audios_features['Label'] = label_encoder.fit_transform(audios_features['Label'])

In [ ]:
# Exclusion des colonnes ' SubClass' et 'IMAGE' de mfcc_features
audios_features = audios_features.drop(columns=['SubClass', 'IMAGE'])

In [ ]:
data = pd.concat([images_features.reset_index(drop=True), audios_features.reset_index(drop=True)], axis=1)

2. La division 2/3 pour l'entrainement et 1/3 pour le test ( en prennant en consédiration 2/3 de chaque sous classe et 1/ de chaque sous classe)

In [ ]:
train_df, test_df = train_test_split(data, test_size=1/3, stratify=data['SubClass'], random_state=42)

In [ ]:
test_df.shape

(5751, 4163)

In [ ]:
train_df.shape

(11501, 4163)

3. On divise ces données d'entrainement en moitié

In [ ]:
train_df_combined, train_df_to_split = train_test_split(train_df, test_size=0.5, stratify=train_df['SubClass'], random_state=42)

In [ ]:
train_df_combined.shape

(5750, 4163)

In [ ]:
train_df_to_split.shape

(5751, 4163)

4. Pour les données combinées, on divise en X_train_combined et y_train_combined

In [ ]:
train_df_combined.head(5)

,SubClass,IMAGE,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,...,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,Label
1249,beach4,beach4-106.png,0.599923,0.0,0.544509,0.281369,1.354041,0.674586,0.0,0.0,...,1.116843,0.502743,0.051246,0.602716,0.321183,0.514368,0.000000,0.0,1.272570,0
11442,restaurant7,restaurant7-238.png,0.889402,0.0,0.270429,0.000000,1.722860,0.751086,0.0,0.0,...,0.666776,0.189546,0.000000,0.662509,0.252713,0.000000,0.163483,0.0,0.475865,7
5206,football1,football1-134.png,0.418690,0.0,0.416819,0.000000,1.559818,0.759001,0.0,0.0,...,0.000000,0.000000,0.000000,0.311041,0.000000,0.485617,0.000000,0.0,0.000000,3
4022,classroom6,classroom6-179.png,0.684965,0.0,0.472656,0.000000,1.538368,0.966992,0.0,0.0,...,0.025651,0.000000,0.242263,0.990658,0.003144,0.403841,0.000000,0.0,0.046537,2
14574,store2,store2-37.png,0.687831,0.0,0.564255,0.000000,1.805864,0.931147,0.0,0.0,...,0.000000,0.912487,0.318709,0.703903,0.158393,0.344479,0.033030,0.0,0.199659,5


In [ ]:
y_train_combined = train_df_combined['Label']
X_train_combined = train_df_combined.iloc[:, 2:train_df_combined.shape[1]-1]

In [ ]:
X_train_combined.columns

Index(['C_0', 'C_1', 'C_2', 'C_3', 'C_4', 'C_5', 'C_6', 'C_7', 'C_8', 'C_9',
       ...
       'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58',
       'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63'],
      dtype='object', length=4160)

5. Pour train_df_to_split, on va avoir trois variables, X_train_lstm, X_train_rn et y_train_for_splited_data

In [ ]:
train_df_to_split.head(5)

,SubClass,IMAGE,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,...,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,Label
9005,jungle,jungle349.png,0.948979,0.0,0.496018,0.0,1.663759,0.637827,0.0,0.0,...,0.000000,0.000000,0.000000,0.833504,0.122834,0.007942,0.000000,0.000000,0.671242,6
12337,river2,river191.png,0.500424,0.0,0.297909,0.0,0.940317,0.954615,0.0,0.0,...,0.729404,0.336445,0.000000,0.305055,0.433456,0.036079,0.000000,0.020387,0.525502,8
5467,football1,football1-37.png,0.603206,0.0,0.653629,0.0,1.665293,0.924123,0.0,0.0,...,0.000000,0.000000,0.432357,0.266740,0.000000,0.729027,0.000000,0.031775,0.000000,3
16,beach,beach112.png,0.475883,0.0,0.594331,0.0,1.277488,0.888983,0.0,0.0,...,0.050469,0.000000,0.184839,0.907827,0.338167,0.704204,0.000000,0.000000,0.769477,0
13794,river7,river7-543.png,0.570892,0.0,0.379372,0.0,1.140499,0.641553,0.0,0.0,...,0.000000,0.394362,0.000000,0.611630,0.000000,0.172866,0.340988,0.000000,0.000000,8


In [ ]:
X_train_rn = train_df_to_split.iloc[:, 2:images_features.shape[1]]
X_train_lstm = train_df_to_split.iloc[:, images_features.shape[1]:train_df_to_split.shape[1]-1]
y_train_for_splited_data = train_df_to_split['Label']

In [ ]:
X_train_rn.columns

Index(['C_0', 'C_1', 'C_2', 'C_3', 'C_4', 'C_5', 'C_6', 'C_7', 'C_8', 'C_9',
       ...
       'C_4086', 'C_4087', 'C_4088', 'C_4089', 'C_4090', 'C_4091', 'C_4092',
       'C_4093', 'C_4094', 'C_4095'],
      dtype='object', length=4096)

In [ ]:
X_train_rn.head(4)

,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,...,C_4086,C_4087,C_4088,C_4089,C_4090,C_4091,C_4092,C_4093,C_4094,C_4095
9005,0.948979,0.0,0.496018,0.0,1.663759,0.637827,0.0,0.0,0.147927,0.0,...,0.354961,0.476827,0.900351,0.609973,0.0,0.0,0.0,0.0,0.351286,0.004927
12337,0.500424,0.0,0.297909,0.0,0.940317,0.954615,0.0,0.0,0.000000,0.0,...,0.198245,0.614126,0.931412,0.710477,0.0,0.0,0.0,0.0,0.445312,0.204972
5467,0.603206,0.0,0.653629,0.0,1.665293,0.924123,0.0,0.0,0.000000,0.0,...,0.087503,0.604430,0.998899,0.740256,0.0,0.0,0.0,0.0,0.207729,0.193277
16,0.475883,0.0,0.594331,0.0,1.277488,0.888983,0.0,0.0,0.000000,0.0,...,0.030596,0.826284,1.026678,0.713945,0.0,0.0,0.0,0.0,0.243224,0.074971


In [ ]:
X_train_lstm.columns

Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
       'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29',
       'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34',
       'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39',
       'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44',
       'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49',
       'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54',
       'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59',
       'feature_60', 'feature_61', 'feature_62', 'feature_63'],
      dtype='object')

In [ ]:
y_train_for_splited_data[:5]

,Label
9005,6
12337,8
5467,3
16,0
13794,8


### 1. Entrainement d'un Réseau de Neurones Dense sur les données visuelles

1. Définition du modèle

In [ ]:
model_image = Sequential()

# Première couche dense
model_image.add(Dense(2048, activation='relu', input_shape=(X_train_rn.shape[1],)))  # 4096 entrées
model_image.add(BatchNormalization())  # Normalisation de lot pour améliorer la convergence
model_image.add(Dropout(0.5))  # Dropout pour éviter le surapprentissage

# Deuxième couche dense
model_image.add(Dense(1024, activation='relu'))
model_image.add(BatchNormalization())  # Normalisation de lot
model_image.add(Dropout(0.4))  # Dropout pour éviter le surapprentissage

# Troisième couche dense
model_image.add(Dense(512, activation='relu'))
model_image.add(BatchNormalization())  # Normalisation de lot
model_image.add(Dropout(0.3))  # Dropout

# Dernière couche pour la classification (softmax pour les étiquettes multiples)
model_image.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output avec le nombre de classes

# Compiling the model
model_image.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Affichage du résumé du modèle
model_image.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 2048)                │       8,390,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 9)                   │           4,617 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,032,585 (42.09 MB)

 Trainable params: 11,025,417 (42.06 MB)

 Non-trainable params: 7,168 (28.00 KB)

In [ ]:
X_train_rn.dtypes.unique()

array([dtype('float64')], dtype=object)

In [ ]:
y_train_for_splited_data.dtypes

dtype('int64')

In [ ]:
# convertir y_train_for_splited_data en int32
#y_train_for_splited_data = np.array(y_train_for_splited_data).astype('int32')

2. Entrainement

In [ ]:
history_rn = model_image.fit(X_train_rn, y_train_for_splited_data, validation_split=0.2, epochs=30, batch_size=32, verbose=1)

Epoch 1/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 38s 244ms/step - accuracy: 0.6792 - loss: 1.0853 - val_accuracy: 0.5899 - val_loss: 3.4286
Epoch 2/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 36s 246ms/step - accuracy: 0.8557 - loss: 0.4968 - val_accuracy: 0.8662 - val_loss: 0.4955
Epoch 3/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 34s 236ms/step - accuracy: 0.8820 - loss: 0.3682 - val_accuracy: 0.8653 - val_loss: 0.4579
Epoch 4/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 42s 241ms/step - accuracy: 0.8944 - loss: 0.3084 - val_accuracy: 0.7428 - val_loss: 0.9907
Epoch 5/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 42s 250ms/step - accuracy: 0.9243 - loss: 0.2509 - val_accuracy: 0.8036 - val_loss: 0.6993
Epoch 6/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 36s 247ms/step - accuracy: 0.9205 - loss: 0.2520 - val_accuracy: 0.9262 - val_loss: 0.2846
Epoch 7/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 35s 243ms/step - accuracy: 0.9264 - loss: 0.2361 - val_accuracy: 0.8540 - val_loss: 0.5689
Epoch 8/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 35s 243ms/step - accuracy: 0.9323 - loss: 0

### 2. Entrainement d'un Réseau de neurones dense sur les données auditives

1. Définition du modèle

In [ ]:
# Définition du modèle sur les caractéristiques LSTM
model_lstm_1 = Sequential()

model_lstm_1.add(Dense(1024, activation='relu', input_shape=(X_train_lstm.shape[1],)))
model_lstm_1.add(BatchNormalization())  # Normalisation de lot
model_lstm_1.add(Dropout(0.4))  # Dropout pour éviter le surapprentissage

model_lstm_1.add(Dense(512, activation='relu'))
model_lstm_1.add(BatchNormalization())  # Normalisation de lot
model_lstm_1.add(Dropout(0.3))  # Dropout

# Dernière couche pour la classification (softmax pour les étiquettes multiples)
model_lstm_1.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output avec le nombre de classes

# Compiling the model
model_lstm_1.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Affichage du résumé du modèle
model_lstm_1.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 1024)                │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 9)                   │           4,617 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 602,121 (2.30 MB)

 Trainable params: 599,049 (2.29 MB)

 Non-trainable params: 3,072 (12.00 KB)

2. Entrainement du modèle

In [ ]:
X_train_lstm.dtypes.unique()

array([dtype('float64')], dtype=object)

In [ ]:
X_train_lstm = np.array(X_train_lstm).astype('float32')

In [ ]:
# Entraînement du modèle
model_lstm_1.fit(X_train_lstm, y_train_for_splited_data, epochs=40, batch_size=32, validation_split=0.2)

Epoch 1/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.3366 - loss: 2.3357 - val_accuracy: 0.3562 - val_loss: 1.9187
Epoch 2/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.4905 - loss: 1.5332 - val_accuracy: 0.4179 - val_loss: 1.6573
Epoch 3/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.5530 - loss: 1.3290 - val_accuracy: 0.5030 - val_loss: 1.3845
Epoch 4/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5745 - loss: 1.2150 - val_accuracy: 0.5291 - val_loss: 1.3004
Epoch 5/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.6050 - loss: 1.1179 - val_accuracy: 0.5474 - val_loss: 1.2905
Epoch 6/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.6412 - loss: 1.0128 - val_accuracy: 0.5456 - val_loss: 1.3756
Epoch 7/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.6796 - loss: 0.9113 - val_accuracy: 0.5317 - val_loss: 1.3648
Epoch 8/40
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.6685 - loss: 0.9217 - val_accu

### 3. Entrainement du modèle de Réseau de neurones dense sur les données combinées

In [ ]:
X_train_combined.dtypes.unique()

array([dtype('float64')], dtype=object)

In [ ]:
# convertir y_train_for_splited_data
X_train_combined = np.array(X_train_combined).astype('float32')

In [ ]:
y_train_combined = np.array(y_train_combined).astype('int32')

1. Définition du modèle

In [ ]:
model_combined = Sequential()

# Première couche dense
model_combined.add(Dense(2048, activation='relu', input_shape=(X_train_combined.shape[1],)))
model_combined.add(BatchNormalization())  # Normalisation de lot pour améliorer la convergence
model_combined.add(Dropout(0.5))  # Dropout pour éviter le surapprentissage

# Deuxième couche dense
model_combined.add(Dense(1024, activation='relu'))
model_combined.add(BatchNormalization())  # Normalisation de lot
model_combined.add(Dropout(0.4))  # Dropout pour éviter le surapprentissage

# Troisième couche dense
model_combined.add(Dense(512, activation='relu'))
model_combined.add(BatchNormalization())  # Normalisation de lot
model_combined.add(Dropout(0.3))  # Dropout

# Dernière couche pour la classification (softmax pour les étiquettes multiples)
model_combined.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output avec le nombre de classes

# Compiling the model
model_combined.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Affichage du résumé du modèle
model_combined.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                      │ (None, 2048)                │       8,521,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 9)                   │           4,617 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,163,657 (42.59 MB)

 Trainable params: 11,156,489 (42.56 MB)

 Non-trainable params: 7,168 (28.00 KB)

In [ ]:
history_combined = model_combined.fit(X_train_combined, y_train_combined, validation_split=0.2, epochs=30, batch_size=32, verbose=1)

Epoch 1/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 38s 242ms/step - accuracy: 0.6822 - loss: 1.0436 - val_accuracy: 0.5678 - val_loss: 2.5244
Epoch 2/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 40s 240ms/step - accuracy: 0.8531 - loss: 0.4676 - val_accuracy: 0.7600 - val_loss: 1.0334
Epoch 3/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 44s 258ms/step - accuracy: 0.8805 - loss: 0.3739 - val_accuracy: 0.9165 - val_loss: 0.3001
Epoch 4/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 38s 238ms/step - accuracy: 0.9096 - loss: 0.2744 - val_accuracy: 0.9043 - val_loss: 0.2672
Epoch 5/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 41s 240ms/step - accuracy: 0.9214 - loss: 0.2406 - val_accuracy: 0.9391 - val_loss: 0.2009
Epoch 6/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 34s 239ms/step - accuracy: 0.9503 - loss: 0.1655 - val_accuracy: 0.9157 - val_loss: 0.2428
Epoch 7/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 34s 237ms/step - accuracy: 0.9361 - loss: 0.1807 - val_accuracy: 0.9035 - val_loss: 0.3531
Epoch 8/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 41s 236ms/step - accuracy: 0.9436 - loss: 0

### Division des données de test

In [ ]:
y_test = test_df['Label']
X_test_combined = test_df.iloc[:, 2:test_df.shape[1]-1]

In [ ]:
X_test_combined

,C_0,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,...,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63
81,0.595019,0.00000,0.481383,0.077948,1.426643,0.844359,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.010167,0.149821,0.414965,0.209141,0.598534,0.000000,0.000000,0.064718
16686,0.936142,0.00000,0.265647,0.000000,1.643624,0.501910,0.0,0.0,0.304374,0.0,...,0.345349,0.278176,0.164386,0.000000,0.736651,0.203419,0.332658,0.000000,0.000000,0.069647
16040,0.428351,0.08283,0.366146,0.000000,1.633000,1.019495,0.0,0.0,0.000000,0.0,...,0.000000,0.210708,0.279752,0.126321,0.953192,0.000000,0.594139,0.238425,0.251097,0.781431
16389,0.707336,0.00000,0.536404,0.000000,1.600991,1.027733,0.0,0.0,0.000000,0.0,...,0.359477,0.372824,0.000000,0.339274,0.312968,0.467156,0.223653,0.000000,0.000000,0.000000
12791,0.630911,0.00000,0.322820,0.108371,1.797607,1.001137,0.0,0.0,0.016860,0.0,...,0.089367,0.319152,0.000000,0.252874,0.219635,0.373061,0.836149,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8057,0.710633,0.00000,0.526511,0.000000,1.188391,0.612820,0.0,0.0,0.000000,0.0,...,0.077954,0.844343,0.307793,0.000000,0.368219,0.301984,0.322507,0.000000,0.125752,0.840060
4857,0.711794,0.00000,0.401048,0.000000,1.519938,1.004501,0.0,0.0,0.000000,0.0,...,0.189582,0.627865,0.786332,0.000000,1.129563,0.051278,0.175464,0.000000,0.100905,0.164808
8636,0.688703,0.00000,0.407750,0.000000,1.582889,0.887991,0.0,0.0,0.000000,0.0,...,0.202369,0.000000,0.000000,0.000000,0.556487,0.462466,0.000000,0.284379,0.000000,0.000000
8173,0.618693,0.00000,0.644669,0.000000,1.520759,0.917463,0.0,0.0,0.000000,0.0,...,0.072477,0.164193,0.000000,0.000000,0.864259,0.887898,0.000000,0.255032,0.000000,0.386974


In [ ]:
X_test_rn = test_df.iloc[:, 2:images_features.shape[1]]
X_test_lstm = test_df.iloc[:, images_features.shape[1]:test_df.shape[1]-1]

In [ ]:
y_test = np.array(y_test).astype('int32')

In [ ]:
X_test_combined = np.array(X_test_combined).astype('float32')

In [ ]:
X_test_lstm = np.array(X_test_lstm).astype('float32')

In [ ]:
X_test_rn = np.array(X_test_rn).astype('float32')

### Evaluation de chaque modèle

###### 1. Modèle entrainé sur les caractéristiques des images

In [ ]:
y_pred_images = model_image.predict(X_test_rn)
y_pred_images_classes = np.argmax(y_pred_images, axis=1)

180/180 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step


In [ ]:
precision = precision_score(y_test, y_pred_images_classes, average='weighted')

recall = recall_score(y_test, y_pred_images_classes, average='weighted')

f1 = f1_score(y_test, y_pred_images_classes, average='weighted')

print(f"Précision: {precision}\n")
print(f"Rappel: {recall}\n")
print(f"F1-Score: {f1}")

Précision: 0.9365928334032555

Rappel: 0.909754825247783

F1-Score: 0.9152731808155133


In [ ]:
print("Rapport de la classification :")
print(classification_report(y_test, y_pred_images_classes, target_names=label_encoder.classes_))

Rapport de la classification :
                precision    recall  f1-score   support

         BEACH       1.00      0.96      0.98       694
          CITY       0.92      0.88      0.90       810
     CLASSROOM       0.97      0.98      0.98       917
FOOTBALL-MATCH       0.99      0.84      0.91       534
        FOREST       0.98      0.98      0.98       454
 GROCERY-STORE       0.98      0.74      0.84       693
        JUNGLE       0.95      0.97      0.96       468
    RESTAURANT       0.56      0.97      0.71       486
         RIVER       0.99      0.89      0.94       695

      accuracy                           0.91      5751
     macro avg       0.93      0.91      0.91      5751
  weighted avg       0.94      0.91      0.92      5751



2. Modèle entrainé sur les caractéristiques extraites à partir des mfcc

In [ ]:
y_pred_audios = model_lstm_1.predict(X_test_lstm)
y_pred_audios_classes = np.argmax(y_pred_audios, axis=1)

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [ ]:
precision = precision_score(y_test, y_pred_audios_classes, average='weighted')

recall = recall_score(y_test, y_pred_audios_classes, average='weighted')

f1 = f1_score(y_test, y_pred_audios_classes, average='weighted')

print(f"Précision: {precision}\n")
print(f"Rappel: {recall}\n")
print(f"F1-Score: {f1}")

Précision: 0.5487746920766267

Rappel: 0.5534689619196661

F1-Score: 0.5499831246101528


In [ ]:
print("Rapport de la classification :")
print(classification_report(y_test, y_pred_audios_classes, target_names=label_encoder.classes_))

Rapport de la classification :
                precision    recall  f1-score   support

         BEACH       0.58      0.64      0.61       694
          CITY       0.47      0.46      0.46       810
     CLASSROOM       0.48      0.44      0.46       917
FOOTBALL-MATCH       0.57      0.63      0.60       534
        FOREST       0.64      0.68      0.66       454
 GROCERY-STORE       0.57      0.60      0.58       693
        JUNGLE       0.87      0.88      0.88       468
    RESTAURANT       0.42      0.33      0.37       486
         RIVER       0.47      0.48      0.48       695

      accuracy                           0.55      5751
     macro avg       0.56      0.57      0.57      5751
  weighted avg       0.55      0.55      0.55      5751



3. Modèle entrainé sur les caractéristiques combinées

In [ ]:
y_pred_combined = model_combined.predict(X_test_combined)
y_pred_combined_classes = np.argmax(y_pred_combined, axis=1)

180/180 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step


In [ ]:
precision = precision_score(y_test, y_pred_combined_classes, average='weighted')

recall = recall_score(y_test, y_pred_combined_classes, average='weighted')

f1 = f1_score(y_test, y_pred_combined_classes, average='weighted')

print(f"Précision: {precision}\n")
print(f"Rappel: {recall}\n")
print(f"F1-Score: {f1}")

Précision: 0.9218075134257423

Rappel: 0.9015823335072162

F1-Score: 0.9010578480237964


In [ ]:
print("Rapport de la classification :")
print(classification_report(y_test, y_pred_images_classes, target_names=label_encoder.classes_))

Rapport de la classification :
                precision    recall  f1-score   support

         BEACH       1.00      0.96      0.98       694
          CITY       0.92      0.88      0.90       810
     CLASSROOM       0.97      0.98      0.98       917
FOOTBALL-MATCH       0.99      0.84      0.91       534
        FOREST       0.98      0.98      0.98       454
 GROCERY-STORE       0.98      0.74      0.84       693
        JUNGLE       0.95      0.97      0.96       468
    RESTAURANT       0.56      0.97      0.71       486
         RIVER       0.99      0.89      0.94       695

      accuracy                           0.91      5751
     macro avg       0.93      0.91      0.91      5751
  weighted avg       0.94      0.91      0.92      5751



### Fusion par moyenne

In [ ]:
def fusion_moyenne(proba_image, proba_audio, proba_combined):

    # Fusion par moyenne des prédictions
    fused_proba = (proba_image + proba_audio + proba_combined) / 3

    # La classe finale (celle avec la probabilité la plus élevée)
    final_predictions = np.argmax(fused_proba, axis=1)

    return final_predictions

In [ ]:
final_predictions = fusion_moyenne(y_pred_images, y_pred_audios, y_pred_combined)

In [ ]:
precision = precision_score(y_test, final_predictions, average='weighted')

recall = recall_score(y_test, final_predictions, average='weighted')

f1 = f1_score(y_test, final_predictions, average='weighted')

print(f"Précision: {precision}\n")
print(f"Rappel: {recall}\n")
print(f"F1-Score: {f1}")

Précision: 0.9585475939348527

Rappel: 0.9570509476612763

F1-Score: 0.9571654872796176


In [ ]:
print("Rapport de la classification :")
print(classification_report(y_test, final_predictions, target_names=label_encoder.classes_))

Rapport de la classification :
                precision    recall  f1-score   support

         BEACH       0.98      0.98      0.98       694
          CITY       0.98      0.89      0.93       810
     CLASSROOM       0.99      0.98      0.98       917
FOOTBALL-MATCH       0.99      0.94      0.97       534
        FOREST       0.99      1.00      0.99       454
 GROCERY-STORE       0.96      0.96      0.96       693
        JUNGLE       0.97      0.99      0.98       468
    RESTAURANT       0.87      0.89      0.88       486
         RIVER       0.88      0.98      0.93       695

      accuracy                           0.96      5751
     macro avg       0.96      0.96      0.96      5751
  weighted avg       0.96      0.96      0.96      5751



### Enragistrement des modèles

In [ ]:
#model_image.save(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_image_hybride_resnet.h5'))
#model_lstm_1.save(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_audio_hybride_resnet.h5'))
#model_combined.save(os.path.join("/content/drive/MyDrive/Multimodale/Model/", "Model - Hybride", 'model_combined_hybride_resnet.h5'))